In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

24/03/03 19:19:44 WARN Utils: Your hostname, MarlonPenuela resolves to a loopback address: 127.0.1.1; using 172.18.73.232 instead (on interface eth0)
24/03/03 19:19:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/03 19:19:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/03 19:19:48 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
spark.version

'3.5.0'

In [5]:
!ls -lh ./data/raw/fhv/2019/10/fhv_tripdata_2019_10.csv.gz

-rw-r--r-- 1 mlops mlops 19M Dec  1  2022 ./data/raw/fhv/2019/10/fhv_tripdata_2019_10.csv.gz


In [12]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True), 
    types.StructField('Affiliated_base_number', types.StringType(), True)
])

In [22]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('./data/raw/fhv/2019/10/fhv_tripdata_2019_10.csv.gz', )

df = df.repartition(6)

df.write.parquet('data/pq/fhv/2019/10/',)


In [74]:
df = spark.read.parquet('data/pq/fhv/2019/10/')

In [23]:
#File size:
!ls -lh ./data/pq/fhv/2019/10/*

-rw-r--r-- 1 mlops mlops    0 Mar  3 19:36 ./data/pq/fhv/2019/10/_SUCCESS
-rw-r--r-- 1 mlops mlops 6.4M Mar  3 19:36 ./data/pq/fhv/2019/10/part-00000-2bb5a947-905a-4b64-a6ec-485cad6af629-c000.snappy.parquet
-rw-r--r-- 1 mlops mlops 6.4M Mar  3 19:36 ./data/pq/fhv/2019/10/part-00001-2bb5a947-905a-4b64-a6ec-485cad6af629-c000.snappy.parquet
-rw-r--r-- 1 mlops mlops 6.4M Mar  3 19:36 ./data/pq/fhv/2019/10/part-00002-2bb5a947-905a-4b64-a6ec-485cad6af629-c000.snappy.parquet
-rw-r--r-- 1 mlops mlops 6.4M Mar  3 19:36 ./data/pq/fhv/2019/10/part-00003-2bb5a947-905a-4b64-a6ec-485cad6af629-c000.snappy.parquet
-rw-r--r-- 1 mlops mlops 6.4M Mar  3 19:36 ./data/pq/fhv/2019/10/part-00004-2bb5a947-905a-4b64-a6ec-485cad6af629-c000.snappy.parquet
-rw-r--r-- 1 mlops mlops 6.4M Mar  3 19:36 ./data/pq/fhv/2019/10/part-00005-2bb5a947-905a-4b64-a6ec-485cad6af629-c000.snappy.parquet


**Q3**: How many taxi trips were there on the 15th of October?
Consider only trips that started on the 15th of October.



In [75]:
from pyspark.sql import functions as F

In [77]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .filter("pickup_date = '2019-10-15'") \
    .count()

62610

In [78]:
df.registerTempTable('fhvhv_2019_10')

In [79]:
spark.sql("""
SELECT
    COUNT(1)
FROM 
    fhvhv_2019_10
WHERE
    to_date(pickup_datetime) = '2019-10-15';
""").show()

+--------+
|count(1)|
+--------+
|   62610|
+--------+



**Q4**: Longest trip for each day
What is the length of the longest trip in the dataset in hours?



In [81]:
df = spark.read.parquet('data/pq/fhv/2019/10/')
df.columns

['dispatching_base_num',
 'pickup_datetime',
 'dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'SR_Flag',
 'Affiliated_base_number']

In [82]:
df.show(5)

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B01236|2019-10-28 12:25:00|2019-10-28 13:50:00|         264|         264|   NULL|                B01236|
|              B02849|2019-10-29 16:22:30|2019-10-29 16:44:41|         264|          63|   NULL|                B02849|
|              B01016|2019-10-30 09:21:11|2019-10-30 09:25:00|         264|          76|   NULL|                B01016|
|              B01362|2019-10-28 05:39:34|2019-10-28 06:02:52|         264|          47|   NULL|                B01362|
|              B03060|2019-10-25 09:39:06|2019-10-25 09:48:32|         264|         178|   NULL|                B03060|
+--------------------+------------------

In [84]:
df \
    .withColumn('duration', (df.dropoff_datetime.cast('long') - df.pickup_datetime.cast('long'))/3600) \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .groupBy('pickup_date') \
        .max('duration') \
    .orderBy('max(duration)', ascending=False) \
    .limit(5) \
    .show()

+-----------+-----------------+
|pickup_date|    max(duration)|
+-----------+-----------------+
| 2019-10-28|         631152.5|
| 2019-10-11|         631152.5|
| 2019-10-31|87672.44083333333|
| 2019-10-01|70128.02805555555|
| 2019-10-17|           8794.0|
+-----------+-----------------+



In [85]:
spark.sql("""
SELECT
    to_date(pickup_datetime) AS pickup_date,
    MAX((CAST(dropoff_datetime AS LONG) - CAST(pickup_datetime AS LONG)) / 3600) AS duration
FROM 
    fhvhv_2019_10
GROUP BY
    1
ORDER BY
    2 DESC
LIMIT 10;
""").show()

+-----------+------------------+
|pickup_date|          duration|
+-----------+------------------+
| 2019-10-28|          631152.5|
| 2019-10-11|          631152.5|
| 2019-10-31| 87672.44083333333|
| 2019-10-01| 70128.02805555555|
| 2019-10-17|            8794.0|
| 2019-10-26| 8784.166666666666|
| 2019-10-30|1464.5344444444445|
| 2019-10-25|1056.8266666666666|
| 2019-10-02| 769.2313888888889|
| 2019-10-23| 745.6166666666667|
+-----------+------------------+



**Q5**: Most frequent `dispatching_base_num`

How many stages this spark job has?



In [44]:
spark.sql("""
SELECT
    dispatching_base_num,
    COUNT(1)
FROM 
    fhvhv_2021_02
GROUP BY
    1
ORDER BY
    2 DESC
LIMIT 5;
""").show()

+--------------------+--------+
|dispatching_base_num|count(1)|
+--------------------+--------+
|              B02510| 3233664|
|              B02764|  965568|
|              B02872|  882689|
|              B02875|  685390|
|              B02765|  559768|
+--------------------+--------+



In [46]:
df \
    .groupBy('dispatching_base_num') \
        .count() \
    .orderBy('count', ascending=False) \
    .limit(5) \
    .show()

+--------------------+-------+
|dispatching_base_num|  count|
+--------------------+-------+
|              B02510|3233664|
|              B02764| 965568|
|              B02872| 882689|
|              B02875| 685390|
|              B02765| 559768|
+--------------------+-------+



**Q6** Least frequent pickup location zone (1 point)


In [86]:
df = spark.read.parquet('data/pq/fhv/2019/10/')

df_zones = spark.read.parquet('zones')

In [87]:
df_zones.columns

['LocationID', 'Borough', 'Zone', 'service_zone']

In [88]:
df.columns

['dispatching_base_num',
 'pickup_datetime',
 'dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'SR_Flag',
 'Affiliated_base_number']

In [90]:
df_zones.registerTempTable('zones')
df.registerTempTable('fhvhv_2019_10')

In [96]:
spark.sql("""
SELECT
    pul.Zone,
    COUNT(1)
FROM 
    fhvhv_2019_10 fhv LEFT JOIN zones pul ON fhv.PULocationID = pul.LocationID
                      LEFT JOIN zones dol ON fhv.DOLocationID = dol.LocationID
GROUP BY 
    1
ORDER BY
    2 ASC
LIMIT 5;
""").show()

+--------------------+--------+
|                Zone|count(1)|
+--------------------+--------+
|         Jamaica Bay|       1|
|Governor's Island...|       2|
| Green-Wood Cemetery|       5|
|       Broad Channel|       8|
|     Highbridge Park|      14|
+--------------------+--------+

